In [ ]:
from utils import network
from utils import visualization
from utils import func
from utils import tissue
import numpy as np
import pandas as pd

#### Load data

In [ ]:
%cd data/
curated_cell_label = "Classical monocytes"
tissue_ns = ['BLD', 'BMA','LIV', 'LNG', 'SPL']
for name in tissue_ns:
    globals()[name] = np.load(f'{name}_{curated_cell_label}_GO_CC_immunesig.npy', allow_pickle = True).tolist()

Sanity checks

In [ ]:
### The filtered gene sets are the same 
method = 'kmeans'
length = len(tissue_ns)
for collection in ['GO-CC', 'immunesig']:
    filter_check = np.array([eval(i).gene_set[collection].filter_index[method] \
                         == eval(j).gene_set[collection].filter_index[method] \
                         for i in tissue_ns for j in tissue_ns]).reshape(length, length)

    if np.all(filter_check):
        print(f'The filtered gene sets of {collection} are the same in each layer')

Combine similarity matrices of GO-CC and Immunesig 

In [ ]:
combine_mat_lst = [func.combine_sim(eval(f'{item}').gene_set['immunesig'].sim_mat[method],\
                                    eval(f'{item}').proj[f'proj_{method}_GO-CC_immunesig'].transpose(),\
                                    eval(f'{item}').gene_set['GO-CC'].sim_mat[method]) for item in tissue_ns]
### filter similarity scores less than 0.1
combine_mat_lst = [func.adj_filter_tr(mat, tr = 0.1) for mat in combine_mat_lst]

In [ ]:
### remove filtered gene sets in the Jaccard layer
num_cores = 60
SPL.call_proj("immunesig", "GO-CC", method = "Jaccard", clustering_metric = "AMI", num_cores = num_cores)
SPL.call_Jaccard("immunesig", num_core = num_cores)
SPL.call_Jaccard("GO-CC", num_core = num_cores)
GO_CC_Jaccard_mat_filtered = SPL.gene_set['GO-CC'].sim_mat['Jaccard'][SPL.gene_set['GO-CC'].filter_index[method],:][:,SPL.gene_set['GO-CC'].filter_index[method]]
immunesig_Jaccard_mat_filtered = SPL.gene_set['immunesig'].sim_mat['Jaccard'][SPL.gene_set['immunesig'].filter_index[method],:][:,SPL.gene_set['immunesig'].filter_index[method]]

In [ ]:
Jaccard_mat_combine = func.combine_sim(immunesig_Jaccard_mat_filtered, \
                  SPL._Jaccard_proj['proj_Jaccard_immunesig_GO-CC'][SPL.gene_set['immunesig'].filter_index[method],:][:,SPL.gene_set['GO-CC'].filter_index[method]],
                  GO_CC_Jaccard_mat_filtered)

In [ ]:
combine_mat_lst.insert(0, Jaccard_mat_combine)

In [ ]:
### markers for gene sets (either Immunesig or GO-CC)
binary_membership = ['#7f7f7f' for item in range(immunesig_Jaccard_mat_filtered.shape[0])] \
+ ['#d62728' for i in range(GO_CC_Jaccard_mat_filtered.shape[0])]

Visualization

In [ ]:
fig_lst = []
vis_lst = []
for adj_tr, ns in zip(combine_mat_lst, tissue_ns):
    fig_temp, vis_temp = visualization.vis_UMAP(adj_tr, binary_membership, title = ns, title_size = 22, \
             UMAP_coord = None, colorbar = False)
    fig_lst.append(fig_temp)
    vis_lst.append(vis_temp)

In [ ]:
### For visualization in Grimon (R package)
M = len(vis_lst)
vis_df = np.concatenate([vis_lst[i].drop(["feature"], axis = 1).to_numpy() for i in range(M)], axis = 1)
color_membership = binary_membership

In [ ]:
import rpy2
### low-level API
import rpy2.rinterface as rinterface
### high-level API 
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

with localconverter(ro.default_converter + pandas2ri.converter):
    vis_df_r = ro.conversion.py2rpy(pd.DataFrame(vis_df))
    vis_label_r = ro.conversion.py2rpy(tissue_ns)
    color_r = ro.conversion.py2rpy(color_membership)
    
### Load into R environment
    ro.globalenv["vis_df_r"] = vis_df_r
    ro.globalenv["vis_label_r"] = vis_label_r
    ro.globalenv["color_r"] = color_r
    ro.r(
        '''
            save(vis_df_r, file = "coexpression/vis_df_r.Rdata")  
            save(vis_label_r, file = "coexpression/vis_label_r.Rdata")
            save(color_r, file = "coexpression/color_r.Rdata")
        ''' 
        )
print ("Transfer successfully!")